<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


1. 네이버 자료

2. 네이버 Place 코드
    * 기준: 서울시 동별, 업종별
    
    
3. 가게정보, 리뷰 정도 겟



# 필요함수 및 라이브러리

In [1]:
import time
import pandas as pd
import numpy as np
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

import warnings
warnings.filterwarnings(action = 'ignore')


## 파일 관련

In [2]:
def get_today():
    import datetime
    now = datetime.datetime.now()
    
    nowDate = now.strftime('%Y-%m-%d')
#     print(nowDate)      # 2015-04-19
 
    nowTime = now.strftime('%H:%M:%S')
#     print(nowTime)      # 12:11:32

    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    return nowDate

In [3]:
import os
def check_file(file_path, file_name):
    return os.path.isfile(file_path+file_name)

In [4]:
def save_df(df, path, file_name):
    df.to_csv(path+file_name, index=False)
    print(f'{path+file_name} 저장 완료')

In [5]:
def read_csv_코드정보(code_path , code_file):
    df = pd.read_csv(code_path + code_file )
    return df

In [6]:
'''음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,
   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, 
   영화공연:CINEMA, 관공서:GOVERNMENT'''

'음식점:DINING, 카페: CAFE, 쇼핑: SHOPPING, 숙박: ACCOMMODATION, 병원의료: HOSPITAL, 은행: BANK,\n   주요소:OIL, 마트슈퍼: MART, 편의점: STORE, 생활편의: CONVENIENCE, 명소: SIGHTS, 체육시설: SPORT, \n   영화공연:CINEMA, 관공서:GOVERNMENT'

###### 서울시 동 load 관련

In [7]:
import pickle
import collections

def get_seoul_dong():
    with open('data/서울시_구별_동정리.pickle','rb') as fw:
        dong_dict = pickle.load( fw)

    dong_key_list = list(dong_dict.keys())
    dong_key_list.sort()
    # dong_key_list

    for 구이름 in dong_key_list:
        dong_dict[구이름] = sorted(dong_dict[구이름])
    
    return dong_dict

In [8]:
dong_dict = get_seoul_dong()

## 웹 조정 관련

In [9]:
def is_clickable(wd):
    return wd.is_enabled() and wd.is_displayed()

In [10]:
def scroll_down(wd,SCROLL_PAUSE_SEC = 1):

    # 스크롤 높이 가져옴
    last_height = wd.execute_script("return document.body.scrollHeight")
#     print(last_height)
    while True:
        # 끝까지 스크롤 다운
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 1초 대기
        time.sleep(SCROLL_PAUSE_SEC)

        # 스크롤 다운 후 스크롤 높이 다시 가져옴
        new_height = wd.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

## 크롤링 에러 방지 관련

In [11]:
def try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        return wd.find_element_by_class_name(class_name).text
    except NoSuchElementException:
        print(col, end=' ')
        return np.NaN
        

In [12]:
def strong_try_none_text_by_class_name(wd,class_name, col='col'):
    try:
        temps = wd.find_elements_by_class_name(class_name)
        for temp in temps:
            if temp.text.startswith('편의'):
                return temp.text.split('\n')[1]
        else:
            return np.NaN
    except NoSuchElementException:
#         print(col, end='')
        return np.NaN
        

In [13]:
def review_dict(s):
    review_dict = {}
#     print(s.split('평균 별점'))
    s = s.split('평균 별점')
    review_dict['평균_별점'] = s[-1].strip()
    s = s[0].split('사진')
    if len(s) >= 2:
        review_dict['사진'] = int(s[-1])
        review_dict['리뷰'] = s[0].split()[-1]
    else:
        review_dict['사진'] = np.NaN
        review_dict['리뷰'] = s[0].split()[-1]
# print(s)
    return review_dict

## get_store_info from naver.place by place_code

In [14]:
def get_store_info(wd, code_no):
    name = wd.find_element_by_class_name('_3XamX').text
#     print(name)
    r_type = try_none_text_by_class_name(wd,'_3ocDE','ty')
#     print(r_type)
    address = try_none_text_by_class_name(wd,'_2yqUQ','ad')
#     print(address)
    tel = try_none_text_by_class_name(wd,'_3ZA0S','tel') 
#     print(tel)
    utility, detail = np.NaN, np.NaN
    for i in wd.find_elements_by_class_name('_1M_Iz'):
#         print(i.text)
        if '편의' in i.text :
            utility = i.text.replace('편의\n','')
        elif '설명' in i.text :
            detail = i.text.replace('설명\n', '')
    
#     print(code_no, utility, detail)
    
    nums_list = wd.find_elements_by_class_name('_1Y6hi')
    try:
        score = nums_list[0].find_element_by_tag_name('em').text
    except:
        score = np.NaN
    try:
        review_num = nums_list[1].find_element_by_tag_name('em').text
    except:
        review_num = np.NaN
        
    try:
        blog_nums = nums_list[2].find_element_by_tag_name('em').text
    except:
        blog_nums = np.NaN
#     print(score, review_num, blog_nums)
    collected_date = get_today()

    # 데이터 렙 펼치기
    for i in wd.find_elements_by_class_name('_3iTUo'):
        if i.text=='더보기':
            i.send_keys('\n')
        else:
            pass
            
    try:
        datalab_1 = {}
        datalab_list = wd.find_elements_by_class_name('_3Ryhx')
        for datalab in datalab_list:
            datalab_1[datalab.find_element_by_class_name('_3hvd9').text] = [x.text for x in datalab.find_elements_by_class_name('_2irYJ') if x.text != '']
    except:
        datalab_1 = np.NaN
        print('d_1', end='')
#     print(name, datalab_1)

    try:
        datalab_g = {}
        datalab_block = wd.find_element_by_class_name('_250cA')
        datalab_age_list = datalab_block.find_elements_by_class_name('_1Nyv0')
        for age_info in datalab_age_list:
            age = age_info.find_element_by_class_name('iO1iA').text
            preference = age_info.find_element_by_class_name('place_blind').text.replace('%','')
            datalab_g[age] = preference
            
        gender_preferance = wd.find_element_by_class_name('c3-chart-arc.c3-target.c3-target-female').text.replace('%',"")
        datalab_g['female_pref'] = gender_preferance
    except:
        datalab_g = np.NaN
        print('d_g', end="")
#     print(datalab_1, datalab_g)

    return name, code_no, r_type, address,tel, score, utility, detail, datalab_1, datalab_g, review_num, blog_nums, collected_date

In [15]:
code_path = 'data/코드/'
구이름 = '강남구'
업종 = 'DINING'
code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'

In [16]:
code_df = read_csv_코드정보(code_path, code_file)

In [17]:
code_df.shape

(6863, 2)

In [18]:
def naver_map_store_scrapping( 구이름, 업종, slow= 1):
    start_time = time.time()

    options = webdriver.ChromeOptions()

    options.add_argument("--start-maximized")
    wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
    wd.implicitly_wait(1*slow)
    
    code_path = 'data/코드/'
    review_path = 'data/리뷰/'
    store_path = 'data/가게/'

    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    review_file = '리뷰정보_' + 구이름 + '_'+ 업종 + '.csv'
    store_file = '가게정보_' + 구이름 + '_'+ 업종 + '.csv'
    
    store_idx = 0
    store_df = pd.DataFrame(columns=("name", "code_no", "r_type","address","tel",
                                     "score","utility","detail","datalab_1", "datalab_g", "review_num",
                                     "blog_num", "collected_date"))
    
    if check_file(store_path, store_file):
        store_df = pd.read_csv(store_path+store_file)
        store_idx = store_df.shape[0]
    done_list = list(store_df['code_no'].apply(lambda x: str(x)).values)
#     print(done_list)
    code_df = pd.read_csv(code_path+code_file, dtype='str')

    print(f'{구이름}  {업종} 업종 의 가게 정보 스크래핑을 시작합니다.')

    print(f'{code_df.shape[0]}개의 가게코드가 있습니다.')
    
    print(f'{store_idx}가 이전에 완료 되었었습니다. 이어서 진행합니다.')

    for _, code_no_ in code_df.iterrows():
        code_no = str(code_no_.values[1])
        if code_no in done_list:
#             print(code_no, '중복중복!!')
            continue
            
        place_url = 'https://pcmap.place.naver.com/restaurant/'+str(code_no)

        wd.get(place_url)
        time.sleep(1*slow)
        scroll_down(wd,0.5)
        cnt_idx = 0

        try:
#             input('ddd')
            wd.implicitly_wait(0.001*slow)
            store_df.loc[store_idx] = get_store_info(wd,code_no)
            wd.implicitly_wait(1*slow)
            print(store_idx, end='#')
            store_idx += 1
            time.sleep(5)
            
        except:
            print(store_idx, end='!!')
            pass
        
        ##################### cooling down
        if store_idx%100 == 0 :
            wd.close()
            wd = webdriver.Chrome(ChromeDriverManager().install(), options= options)  # chromedriver 열기
            wd.implicitly_wait(1*slow)
            time.sleep(1*slow)
            
        ##################### save just in case:
        if store_idx%30 == 0 :
            store_df.to_csv(store_path+store_file, index=False)
            print(구이름, 업종, f'가게 {store_idx}개 중간저장')
            print(f'{time.time()-start_time} 소요')
#         ##################### Test Test Test
#         if store_idx%10==0:
#             store_df.to_csv(store_path+store_file, index=False)
#             print(구이름,업종, f'가게 {store_idx}개 완료' )
#             print(f'{time.time()-start_time} 소요')
#             wd.close()
#             break
    else:
        store_df.to_csv(store_path+store_file, index=False)
        print(구이름, 업종, f'가게 {store_idx}개 완료' )
        print(f'{time.time()-start_time} 소요')
        wd.close()
        
#     return store_df

# 셋팅

In [19]:
# code_path = 'data/코드/'
# review_path = 'data/리뷰/'
# store_path = 'data/가게/'

# code_file = '가게코드_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# review_file = '리뷰정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 + '_' + 동이름+ '_'+ 업종 + '.csv'

In [20]:
code_path = 'data/코드/'
store_path = 'data/가게/'
# code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
# store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'

In [21]:
detail= True
slow=1

#  코드 수집 및 저장

In [22]:
업종= 'DINING'

for 구이름 in sorted(dong_dict.keys()):
    code_file = '가게코드_' + 구이름 + '_'+ 업종 + '.csv'
    if check_file(code_path, code_file):
        store_file = '가게정보_' + 구이름 +  '_'+ 업종 + '.csv'
        naver_map_store_scrapping( 구이름, 업종, slow= 1)
    else:
        print(f'{구이름} {업종}이 없습니다. ')

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


강남구  DINING 업종 의 가게 정보 스크래핑을 시작합니다.
6863개의 가게코드가 있습니다.
15가 이전에 완료 되었었습니다. 이어서 진행합니다.
15#16#17#강남구 DINING 가게 18개 중간저장
26.250889778137207 소요
18#19!!19#

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 95.0.4638
[WDM] - Get LATEST driver version for 95.0.4638
[WDM] - Driver [C:\Users\bigne\.wdm\drivers\chromedriver\win32\95.0.4638.17\chromedriver.exe] found in cache


20!!강남구 DINING 가게 20개 완료
43.11222553253174 소요
강동구 DINING이 없습니다. 
강북구 DINING이 없습니다. 
강서구 DINING이 없습니다. 
관악구 DINING이 없습니다. 
광진구 DINING이 없습니다. 
구로구 DINING이 없습니다. 
금천구 DINING이 없습니다. 
노원구 DINING이 없습니다. 
도봉구 DINING이 없습니다. 
동대문구 DINING이 없습니다. 
동작구 DINING이 없습니다. 
마포구 DINING이 없습니다. 
서대문구 DINING이 없습니다. 
서초구 DINING이 없습니다. 
성동구 DINING이 없습니다. 
성북구 DINING이 없습니다. 
송파구 DINING이 없습니다. 
양천구 DINING이 없습니다. 
영등포구 DINING이 없습니다. 
용산구 DINING이 없습니다. 
은평구  DINING 업종 의 가게 정보 스크래핑을 시작합니다.
3261개의 가게코드가 있습니다.
0가 이전에 완료 되었었습니다. 이어서 진행합니다.
0#

KeyboardInterrupt: 

In [23]:
df = pd.read_csv('data/가게/가게정보_강남구_DINING.csv')

In [24]:
df.shape

(20, 13)